In [1]:
import idx2numpy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# ---------------------------------
# IMPORT MNIST DATASET
# ---------------------------------

imagefile = '/Users/mellebrouwer/Programmeren/Github/mnist_cnn/data/t10k-images-idx3-ubyte'
imagearray = idx2numpy.convert_from_file(imagefile)

labelfile = '/Users/mellebrouwer/Programmeren/Github/mnist_cnn/data/t10k-labels-idx1-ubyte'
labelarray = idx2numpy.convert_from_file(labelfile)

X = imagearray
y = labelarray

# plt.imshow(imagearray[1], cmap="binary")
# plt.show()

In [3]:
# ---------------------------------
# PREPARE & SPLIT DATASETS
# ---------------------------------

# Reshape 3D to 2D
X = X.reshape((len(X), 28*28), order='F')

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [29]:
# ---------------------------------
# SIMPLE SVM BASELINE
# ---------------------------------

model = SGDClassifier(random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)

acc = accuracy_score(y_test, preds)
mcm = multilabel_confusion_matrix(y_test, preds)

print("\nSVM BASELINE")
print(f"Accuracy: {acc}")


SVM BASELINE
Accuracy: 0.8816


In [28]:
# ---------------------------------
# SIMPLE NEURAL NET BASELINE
# ---------------------------------

# Input shape & random state
tf.random.set_seed(0)
_, input_shape = X_train.shape

# Build model
model = keras.Sequential([
                          keras.Input(shape=(input_shape,)),
                          layers.Dense(128, activation='relu'),
                          layers.Dense(128, activation='relu'),
                          layers.Dense(10, activation="softmax"),
                          ])

# Compile model
model.compile(
             optimizer="rmsprop",
             loss="sparse_categorical_crossentropy",
             metrics=["sparse_categorical_accuracy"],
             )

# Set early stopping
early_stop = [keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    min_delta=1e-2,
                    patience=3,
                    verbose=1,
                    )]
    
# Train model
history = model.fit(
                    X_train,
                    y_train,
                    batch_size=64,
                    epochs=30,
                    callbacks=early_stop,
                    validation_split=0.2,
                    )

# Test model
preds = model.predict(X_test)

# Conver softmax probabilities to integers
preds = np.argmax(preds,axis=-1)

acc = accuracy_score(y_test, preds)
mcm = multilabel_confusion_matrix(y_test, preds)

print("\nNEURAL NET BASELINE")
print(f"Accuracy: {acc}")


Epoch 1/30
94/94 [==============================] - 1s 5ms/step - loss: 18.4229 - sparse_categorical_accuracy: 0.6264 - val_loss: 2.4910 - val_sparse_categorical_accuracy: 0.8573
Epoch 2/30
94/94 [==============================] - 0s 3ms/step - loss: 2.2004 - sparse_categorical_accuracy: 0.8767 - val_loss: 2.2103 - val_sparse_categorical_accuracy: 0.8620
Epoch 3/30
94/94 [==============================] - 0s 3ms/step - loss: 1.2908 - sparse_categorical_accuracy: 0.9100 - val_loss: 1.9250 - val_sparse_categorical_accuracy: 0.8913
Epoch 4/30
94/94 [==============================] - 0s 3ms/step - loss: 0.6959 - sparse_categorical_accuracy: 0.9399 - val_loss: 2.0572 - val_sparse_categorical_accuracy: 0.8867
Epoch 5/30
94/94 [==============================] - 0s 3ms/step - loss: 0.6157 - sparse_categorical_accuracy: 0.9542 - val_loss: 1.7530 - val_sparse_categorical_accuracy: 0.9120
Epoch 6/30
94/94 [==============================] - 0s 3ms/step - loss: 0.3790 - sparse_categorical_accuracy:

In [57]:
# ---------------------------------------------
# PREPARE & SPLIT DATASETS RESHAPE TO ndim=4
# ---------------------------------------------

# Reassign original arrays
X = imagearray.reshape(-1, 28, 28, 1)
y = labelarray

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [62]:
# ---------------------------------
# CONVOLUTIONAL NEURAL NETWORK
# ---------------------------------

# Input shape & random state
tf.random.set_seed(0)

# Build model
model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# Compile model
model.compile(
             optimizer="rmsprop",
             loss="sparse_categorical_crossentropy",
             metrics=["sparse_categorical_accuracy"],
             )

# Set early stopping
early_stop = [keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    min_delta=1e-2,
                    patience=3,
                    verbose=1,
                    )]
    
# Train model
history = model.fit(
                    X_train,
                    y_train,
                    batch_size=64,
                    epochs=30,
                    callbacks=early_stop,
                    validation_split=0.2,
                    )

# Test model
preds = model.predict(X_test)

# Conver softmax probabilities to integers
preds = np.argmax(preds,axis=-1)

acc = accuracy_score(y_test, preds)
mcm = multilabel_confusion_matrix(y_test, preds)

print("\nCONVOLUTIONAL NEURAL NETWORK")
print(f"Accuracy: {acc}")


Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 128)             